In [1]:
import cv2
import os
import pandas as pd

In [25]:
def create_dataframe_from_directory(directory):
    """
    Vytvorenie Pandas DataFrame obsahujúceho cesty k súborom a štítky z adresárovej štruktúry.

    Args:
    - (str): Adresár obsahujúci podadresáre so súbormi obrázkov.

    Returns::
    - df (DataFrame): Pandas DataFrame so stĺpcami "file_path" a "label".
    """
    
    data = []

    for label, subdir in enumerate(["normal", "glaucoma"]):
        subdir_path = os.path.join(directory, subdir)
        print(subdir_path)
        for filename in os.listdir(subdir_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                file_path = os.path.join(subdir_path, filename)
                data.append((file_path, label))

    df = pd.DataFrame(data, columns=["file_path", "label"])
    
    return df

In [26]:
# Example usage:
dir = "RIM-ONE/RIM-ONE/NOT PARTITIONED"

df = create_dataframe_from_directory(dir)

RIM-ONE/RIM-ONE/NOT PARTITIONED\normal
RIM-ONE/RIM-ONE/NOT PARTITIONED\glaucoma


In [27]:
df.label.value_counts()

label
0    313
1    172
Name: count, dtype: int64

In [15]:
df.head()

,file_path,label
0,ACRIMA\ACRIMA\NOT PARTITIONED\normal\Im001_ACR...,0
1,ACRIMA\ACRIMA\NOT PARTITIONED\normal\Im002_ACR...,0
2,ACRIMA\ACRIMA\NOT PARTITIONED\normal\Im003_ACR...,0
3,ACRIMA\ACRIMA\NOT PARTITIONED\normal\Im004_ACR...,0
4,ACRIMA\ACRIMA\NOT PARTITIONED\normal\Im005_ACR...,0


In [6]:
def apply_clahe (df, output_dir, clip_limit=2.0, tile_grid_size=(8, 8)):
    """
    Aplikujte adaptívne vyrovnanie histogramu s obmedzeným kontrastom (CLAHE) na obrázky v DataFrame a uloží vylepšené obrázky do výstupného adresára.

    Args:
    - df (DataFrame): Pandas DataFrame obsahujúci cesty k súborom a štítky.
    - output_dir (str): Adresár na uloženie rozšírených obrázkov.
    - clip_limit (float): Prahová hodnota pre obmedzenie kontrastu.
    - tile_grid_size (tuple): Veľkosť mriežky na vyrovnávanie histogramu.

    Returns:
    - roi_df (DataFrame): Pandas DataFrame obsahujúci cesty vylepšených obrázkov a ich zodpovedajúce značky.
    """

    os.makedirs(output_dir, exist_ok=True)    
    roi_rows = []
    
    for index, row in df.iterrows():   
        if row['label'] == 0:
            output_dir_roi = os.path.join(output_dir, 'normal_onh')
            os.makedirs(output_dir_roi, exist_ok=True)  
        elif row['label'] == 1:
            output_dir_roi = os.path.join(output_dir, 'glaucoma_onh')
            os.makedirs(output_dir_roi, exist_ok=True)
            
        image = cv2.imread(row['file_path'])
            
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)    
        l, a, b = cv2.split(lab)   
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
        l_clahe = clahe.apply(l)   
        lab_clahe = cv2.merge((l_clahe, a, b))  
        clahe_image = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2RGB)
        
        roi_filename = os.path.basename(row['file_path']).split('.')[0] + '.jpg'
        roi_path = os.path.join(output_dir_roi, roi_filename)
        cv2.imwrite(roi_path, cv2.cvtColor(clahe_image, cv2.COLOR_BGR2RGB))
        
        roi_rows.append({'roi_path': roi_path, 'label': row['label']})
    
    roi_df = pd.DataFrame(roi_rows)
    
    return roi_df

In [28]:
clahe_df = apply_clahe(df, 'clahe_images_cropped')